In [3]:
from SPARQLWrapper import SPARQLWrapper, JSON
from rdflib import Graph
import folium
from shapely import wkt
import os
import openai

In [4]:
graph = Graph()

# specify path to datasets
path_to_aqqa_kg = "/mnt/data/AQQA_graph/AQQA_KG.ttl"
graph.parse(path_to_aqqa_kg, format='ttl')

<Graph identifier=Na76b342eae314337a9ec515e59359f53 (<class 'rdflib.graph.Graph'>)>

In [11]:
# read sparql query
path_to_sparql_query = "../../scripts/sparql_queries/get_all_cams_aq_values.ttl"
with open(path_to_sparql_query, 'r') as query_file:
    query_string = query_file.read()

In [12]:
for row in graph.query(query_string):
    print(row)

(rdflib.term.Literal('583.2448', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')), rdflib.term.Literal('2020-01-01', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#date')), rdflib.term.URIRef('http://example.com/ontologies/aqqa#Cell_24_53'))
(rdflib.term.Literal('415.77603', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')), rdflib.term.Literal('2020-01-02', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#date')), rdflib.term.URIRef('http://example.com/ontologies/aqqa#Cell_24_53'))
(rdflib.term.Literal('523.3698', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')), rdflib.term.Literal('2020-01-03', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#date')), rdflib.term.URIRef('http://example.com/ontologies/aqqa#Cell_24_53'))
(rdflib.term.Literal('187.28386', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')), rdflib.term.Literal('2020-01-04', datatype=rdflib.term.U

#### Text to Sparql using GPT-turbo

In [12]:
# set environment variable
#os.environ["OPENAI_KEY"] = "<OPEN AI KEY>"

In [13]:
openai.api_key = os.environ["OPENAI_KEY"]

In [14]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message["content"]

In [20]:
prompt = """
Your task is to transform natural language to a sparql query. 
There are the following labels for observable attributes: CO, NO2, O3, PM10, PM2P5 SO2
The full names of those attributes are: carbon_monoxide nitrogen_dioxide ozone particulate_matter_10um particulate_matter_2.5um sulphur_dioxide
For the query always the short version has to be used.
If you are not sure if you can construct the sparql query, respond <I am sorry, but I cannot create a sparql query from this question>
The structure of this query is purposely divided in subqueries to improve performance. Don't change this structure.

<Natural Language>: What was the <observable attribute> concentration in <location name> for <month> <year>?

<Sparql Query>: 
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#> 
PREFIX gadm: <http://example.com/ontologies/gadm#>

SELECT ?obs_result ?obs_time ?foi_ent
WHERE {
    {
        SELECT ?foi_ent
        WHERE {
            ?foi_ent a sosa:FeatureOfInterest ;
                geo:intersects ?gadm_ent .
            ?gadm_ent a gadm:AdministrativeUnit ;
                gadm:hasName <location name> ;
                gadm:hasNationalLevel 3 ;
        } 
    }

    ?obs_ent a sosa:Observation ;
            sosa:hasSimpleResult ?obs_result ; 
            sosa:resultTime ?obs_time ;
            sosa:hasFeatureOfInterest ?foi_ent ;
            sosa:observedProperty ?obs_prop_ent .
    ?obs_prop_ent a sosa:ObservableProperty ;
        rdfs:label <observable attribute e.g. PM10> .

    FILTER (YEAR(?obs_time) = <year> && MONTH(?obs_time) = <mont>)

<Natural Language>: Which municipalities exceeded the <observable attribute> threshold of <threshold value> for the year <year> and month <month>?

<Sparql Query>: 
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#> 
PREFIX gadm: <http://example.com/ontologies/gadm#>

SELECT ?gadm_name ?obs_result ?obs_time 
WHERE {
    {
        SELECT ?foi_ent ?cell_geom ?gadm_name
        WHERE {
            ?foi_ent a sosa:FeatureOfInterest ;
                geo:intersects ?gadm_ent ;
                geo:hasGeometry ?cell_geom_ent .
            ?cell_geom_ent geo:asWKT ?cell_geom .
            ?gadm_ent a gadm:AdministrativeUnit ;
                gadm:hasName ?gadm_name;
                gadm:hasNationalLevel 3 ;
        } 
    }

    ?obs_ent a sosa:Observation ;
            sosa:hasSimpleResult ?obs_result ; 
            sosa:resultTime ?obs_time ;
            sosa:hasFeatureOfInterest ?foi_ent ;
            sosa:observedProperty ?obs_prop_ent .
    ?obs_prop_ent a sosa:ObservableProperty ;
        rdfs:label <observable attribute e.g. CO> .

    FILTER (YEAR(?obs_time) = <year> && MONTH(?obs_time) = <month>)
    FILTER (?obs_result > <threshold value>)
}

} 

<Natural Language>: List all observations where the ozone value was above 40 during summer 2020.

"""
response = get_completion(prompt)
print(response)

<Sparql Query>: 
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#> 
PREFIX gadm: <http://example.com/ontologies/gadm#>

SELECT ?obs_result ?obs_time ?foi_ent
WHERE {
    {
        SELECT ?foi_ent
        WHERE {
            ?foi_ent a sosa:FeatureOfInterest ;
                geo:intersects ?gadm_ent .
            ?gadm_ent a gadm:AdministrativeUnit ;
                gadm:hasName <location name> ;
                gadm:hasNationalLevel 3 ;
        } 
    }

    ?obs_ent a sosa:Observation ;
            sosa:hasSimpleResult ?obs_result ; 
            sosa:resultTime ?obs_time ;
            sosa:hasFeatureOfInterest ?foi_ent ;
            sosa:observedProperty ?obs_prop_ent .
    ?obs_prop_ent a sosa:ObservableProperty ;
        rdfs:label "ozone" .

    FILTER (YEAR(?obs_time) = 2020 && MONTH(?obs_time) >= 6 && MONTH(?obs_time) <= 8)
    FILTER (?obs_result > 40)
}


In [24]:
response[15:]

' \nPREFIX sosa: <http://www.w3.org/ns/sosa/>\nPREFIX geo: <http://www.opengis.net/ont/geosparql#> \nPREFIX gadm: <http://example.com/ontologies/gadm#>\n\nSELECT ?obs_result ?obs_time ?foi_ent\nWHERE {\n    {\n        SELECT ?foi_ent\n        WHERE {\n            ?foi_ent a sosa:FeatureOfInterest ;\n                geo:intersects ?gadm_ent .\n            ?gadm_ent a gadm:AdministrativeUnit ;\n                gadm:hasName <location name> ;\n                gadm:hasNationalLevel 3 ;\n        } \n    }\n\n    ?obs_ent a sosa:Observation ;\n            sosa:hasSimpleResult ?obs_result ; \n            sosa:resultTime ?obs_time ;\n            sosa:hasFeatureOfInterest ?foi_ent ;\n            sosa:observedProperty ?obs_prop_ent .\n    ?obs_prop_ent a sosa:ObservableProperty ;\n        rdfs:label "ozone" .\n\n    FILTER (YEAR(?obs_time) = 2020 && MONTH(?obs_time) >= 6 && MONTH(?obs_time) <= 8)\n    FILTER (?obs_result > 40)\n}'

In [26]:
query = """
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#> 
PREFIX gadm: <http://example.com/ontologies/gadm#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

# Aggregating option should be MIN/MAX/AVG
SELECT (AVG(?obs_result) AS ?avg_obs_result) 
WHERE {
    {
        SELECT ?foi_ent
        WHERE {
            ?foi_ent a sosa:FeatureOfInterest ;
                geo:intersects ?gadm_ent .
            ?gadm_ent a gadm:AdministrativeUnit ;
                # gadm name is variable
                gadm:hasName 'Linz' ;
                # how to handle national level? Default 3 if possible?
                # aggregate by national level of more than 1 is available?
                gadm:hasNationalLevel 3 ;
        } 
    }

    ?obs_ent a sosa:Observation ;
            sosa:hasSimpleResult ?obs_result ; 
            sosa:resultTime ?obs_time ;
            sosa:hasFeatureOfInterest ?foi_ent ;
            sosa:observedProperty ?obs_prop_ent .
    ?obs_prop_ent a sosa:ObservableProperty ;
        # observable property is variable 
        rdfs:label 'CO' .

    # Filtering should be possible for a whole year, a year and a month or a specific time range
    # Possibility to filter by season? 
    FILTER (YEAR(?obs_time) = 2020 && MONTH(?obs_time) = 1)
} 
"""

In [27]:
for row in graph.query(query):
    print(row)

(rdflib.term.Literal('357.04685883870985', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')),)
